In [40]:
import pandas as pd                                     #导包
import numpy as np
import re
N = 100000
res=pd.read_csv('res.csv')                              #读取属性数据集
df=pd.read_csv('df.csv')                                #读取规则数据集
df_A = df[(df['target']=='A')]                          #读取属性数据集中target=A的数据集
len_A = len(df_A)                                       #计算target=A的数据集的数量
len_B = 100000-len_A                                    #计算target=B的数据集的数量

In [41]:
###################按照相互等价的为一类分类###########################
rules = res.ix[:,1]                                     #提取规则数据集中的规则列
new_rules = []                                          #创建一个list存放计算过后的规则
reExp1 = r'X(.*?)=(.*?)[,}]'                            #正则表达式过滤条件(提取等号两边的数值，组成一个tuple)
for tup in rules:                                       #遍历所有规则------------>复杂度O(R)
    invert = []                                         #创建一个list存放正则后的结果
    invert = re.compile(reExp1).findall(tup)
    tmp = map(lambda x:int(x[0]) + int(x[1]),invert)    #将每个tuple中的数值相加
    tmp = list(tmp)                                     #将结果转成list类型
    tmp.sort()                                          #将list中数值从小到大排序
    tmp = "".join([str(i) for i in tmp])                #将数值合并成一个数字，(例如[2,8,10]-->[2810])
    new_rules.append(tmp)                               #将结果添加到new_rules
dic_rul = dict(zip((res.ix[:,0]),new_rules))            #新建一个字典，key=规则序号，values=新的rules
rev_dic = {}                                            #新建一个字典存放等价规则分类后的规则，key=rules，values=具有相同rules的rules序号                                     
for k,v in dic_rul.items():                             #遍历dic---------------->复杂度O(R)
    rev_dic.setdefault(v, []).append(k)                 #将相同规则的序号放到一起--->复杂度O(1)

In [52]:
pd.DataFrame.from_dict(rev_dic,orient='index')

,0,1,2,3,4,5,6,7,8,9,...,115,116,117,118,119,120,121,122,123,124
31114,15528,15561.0,16104.0,16140.0,16257.0,16279.0,16281.0,16812.0,17282.0,17284.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71017,5349,5364.0,5366.0,5368.0,5578.0,5587.0,5600.0,5603.0,5893.0,5895.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61019,5196,5202.0,5542.0,5549.0,5650.0,5657.0,6044.0,6050.0,6555.0,6562.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6917,13784,13815.0,13816.0,13824.0,13830.0,13902.0,13904.0,13906.0,14518.0,14519.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
517,854,863.0,870.0,897.0,1036.0,1038.0,1040.0,1493.0,1495.0,1497.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3813,9530,9562.0,10305.0,10332.0,10333.0,10345.0,10463.0,11285.0,11299.0,11337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2515,3489,4200.0,4367.0,30629.0,30918.0,31433.0,31458.0,32651.0,32823.0,39935.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111115,3048,3055.0,3076.0,4513.0,4534.0,4732.0,15925.0,15932.0,15937.0,15954.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121419,22139,22142.0,22143.0,22162.0,22167.0,22178.0,22266.0,22292.0,22407.0,22411.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3817,17593,17623.0,18132.0,18134.0,18135.0,18327.0,18405.0,18443.0,26535.0,26561.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
dic_sup = dict(zip((res.ix[:,0]),res.ix[:,2]))                     #创建字典存放sup
rul = []                                                           #新建list存放规则
interest =[]                                                       #新建list存放每个类最大的interest值
for k,v in rev_dic.items():                                        #
    NN = []                                                        #新建list存放每个key中的所有interest值
    for j in v:                                                    #循环每个key对应的values--->前两个循环O(R)
        sup = dic_sup[j]
        inter = ((N*sup)*(N-N*sup))/(len_A*len_B)
        NN.append(inter)
    K = max(NN)
    interest.append(K)
    rul.append(rules[v[NN.index(max(NN))]])
result = pd.DataFrame({'interest':interest,'rules':rul},columns=['rules','interest'])

In [46]:
result.to_csv('E:\\work\\CISDI\\data\\result1.csv')